<a href="https://colab.research.google.com/github/Janemura/CTF-Challenges/blob/main/cybersecurity_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import ipywidgets as widgets
from IPython.display import display

# 1. Simulate Network Traffic Data with Specific Suspicious Patterns
np.random.seed(42)

# Define a list of suspicious websites (these would be real suspicious sites in a real-world system)
suspicious_sites = ['malicious.com', 'badwebsite.net', 'phishing.org', 'suspicious.io']

# Create normal traffic (10 entries) with non-suspicious websites
normal_data = pd.DataFrame({
    'src_ip': [f"192.168.1.{i}" for i in range(1, 11)],
    'dst_ip': [f"10.0.0.{i}" for i in range(1, 11)],
    'packet_size': np.random.randint(100, 500, size=10),   # smaller packet sizes for normal traffic
    'duration': np.random.uniform(0.1, 1.9, size=10),      # shorter duration for normal traffic
    'site_visited': ['google.com', 'facebook.com', 'youtube.com', 'twitter.com', 'linkedin.com',
                     'example.com', 'wikipedia.org', 'yahoo.com', 'amazon.com', 'bing.com'],
    'flagged': 0  # 0 indicates normal traffic
})

# Create suspicious traffic (10 entries) with suspicious websites
suspicious_data = pd.DataFrame({
    'src_ip': [f"192.168.1.{i}" for i in range(11, 21)],
    'dst_ip': [f"10.0.0.{i}" for i in range(11, 21)],
    'packet_size': np.random.randint(600, 1500, size=10),  # larger packet sizes for suspicious traffic
    'duration': np.random.uniform(2.0, 5.0, size=10),      # longer duration for suspicious traffic
    'site_visited': ['malicious.com', 'badwebsite.net', 'phishing.org', 'suspicious.io',
                     'malicious.com', 'phishing.org', 'badwebsite.net', 'suspicious.io',
                     'malicious.com', 'badwebsite.net'],
    'flagged': 1  # 1 indicates suspicious traffic
})

# Combine normal and suspicious traffic into one dataset with 20 entries
data = pd.concat([normal_data, suspicious_data], ignore_index=True)

# 2. Flag Traffic Based on Suspicious Sites Visited
def flag_based_on_sites(row):
    # Flag as suspicious if the site visited is in the list of suspicious sites
    if row['site_visited'] in suspicious_sites:
        return 1  # suspicious
    else:
        return 0  # normal

# Apply the site-based flagging
data['flagged'] = data.apply(flag_based_on_sites, axis=1)

# 3. Split Data and Train the Model
# Prepare features and labels
X = data[['packet_size', 'duration']]
y = data['flagged']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train the RandomForest Classifier
model = RandomForestClassifier(n_estimators=50, random_state=42)
model.fit(X_train, y_train)

# Evaluate accuracy
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

# 4. Print the model accuracy first
print("Model accuracy:", accuracy)

# 5. Define Action Functions
def block_ip(ip):
    print(f"Simulated block action: IP {ip} blocked.")

def alert_admin(ip):
    print(f"Simulated alert: Admin notified of suspicious activity from IP {ip}.")

# 6. Real-Time Monitoring Simulation with Threat Detection and Explanations
def monitor_and_act():
    print("\nMonitoring simulated network traffic...")
    for i, row in data.iterrows():
        ip = row['src_ip']
        features = row[['packet_size', 'duration']].values.reshape(1, -1)
        prediction = model.predict(features)[0]

        # Initialize a list to store reasons for suspicion
        reasons = []

        # Check if packet size is above the threshold (suspicious traffic if > 600 bytes)
        if row['packet_size'] > 600:
            reasons.append("Large packet size (>600 bytes)")

        # Check if duration is suspiciously long (greater than 2 seconds)
        if row['duration'] > 2.0:
            reasons.append("Long duration (>2 seconds)")

        # Check if the site visited is suspicious
        if row['site_visited'] in suspicious_sites:
            reasons.append(f"Suspicious site visited: {row['site_visited']}")

        if prediction == 1 or len(reasons) > 0:  # Threat detected (model or manual suspicion)
            print(f"Suspicious traffic detected from IP {ip}. Reasons: {', '.join(reasons)}. Taking action...")
            block_ip(ip)
            alert_admin(ip)
        else:
            print(f"Traffic from IP {ip} is normal.")

# 7. Interactive User Interface Using ipywidgets
# Interactive interface for manual control
ip_input = widgets.Text(description="IP Address:")
block_button = widgets.Button(description="Block IP")
alert_button = widgets.Button(description="Alert Admin")

# Button event handlers
def on_block_button_click(b):
    ip = ip_input.value
    block_ip(ip)

def on_alert_button_click(b):
    ip = ip_input.value
    alert_admin(ip)

# Link buttons with event handlers
block_button.on_click(on_block_button_click)
alert_button.on_click(on_alert_button_click)

# Display interface for manual actions
display(ip_input, block_button, alert_button)

# Run monitoring simulation
monitor_and_act()  # Corrected to remove "do" keyword


Model accuracy: 1.0


Text(value='', description='IP Address:')

Button(description='Block IP', style=ButtonStyle())

Button(description='Alert Admin', style=ButtonStyle())


Monitoring simulated network traffic...
Traffic from IP 192.168.1.1 is normal.
Traffic from IP 192.168.1.2 is normal.
Traffic from IP 192.168.1.3 is normal.
Traffic from IP 192.168.1.4 is normal.
Traffic from IP 192.168.1.5 is normal.
Traffic from IP 192.168.1.6 is normal.
Traffic from IP 192.168.1.7 is normal.
Traffic from IP 192.168.1.8 is normal.
Traffic from IP 192.168.1.9 is normal.
Traffic from IP 192.168.1.10 is normal.
Suspicious traffic detected from IP 192.168.1.11. Reasons: Large packet size (>600 bytes), Long duration (>2 seconds), Suspicious site visited: malicious.com. Taking action...
Simulated block action: IP 192.168.1.11 blocked.
Simulated alert: Admin notified of suspicious activity from IP 192.168.1.11.
Suspicious traffic detected from IP 192.168.1.12. Reasons: Large packet size (>600 bytes), Long duration (>2 seconds), Suspicious site visited: badwebsite.net. Taking action...
Simulated block action: IP 192.168.1.12 blocked.
Simulated alert: Admin notified of suspi

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X